In [1]:
!pip install -q torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121
!pip install -q matplotlib datasets ftfy sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.6 MB/s eta 0:00:00


In [2]:
import os
import torch
import torch.nn as nn
import math
import torch.profiler as profiler # Ver en qué capas se consume más memoria GPU o tiempo.
import matplotlib.pyplot as plt
from datasets import load_dataset
import re
import ftfy
import unicodedata
from pathlib import Path
import sentencepiece as spm
import os
from datasets import DatasetDict
import torch
import torch.nn as nn
import torch.nn.functional as F
import math



In [3]:
print("GPU available:", torch.cuda.is_available())

GPU available: True


In [4]:
def evaluate_ppl(model, dataloader, loss_fn, device):
    model.eval() # Le dice al modelo que se comporte como inferencia
    total_loss = 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            loss = loss_fn(logits.view(-1, logits.size(-1)), y.view(-1))
            total_loss += loss.item()
    avg_loss = total_loss / len(dataloader)
    ppl = math.exp(avg_loss)
    return avg_loss, ppl

In [5]:
TOK_MODEL_PATH = "./resources/models/bpe_model_shakespeare.model"
DATASET_PATH = "./resources/datasets/tinyshakespeare.txt"
train_path = Path("./resources/datasets/shakespeare_clean_train.txt")
test_path = Path("./resources/datasets/shakespeare_clean_test.txt")
valid_path = Path("./resources/datasets/shakespeare_clean_validation.txt")

def light_clean_fn(example):
    t = example["text"]
    t = ftfy.fix_text(t)
    t = unicodedata.normalize("NFKC", t)
    t = (t.replace("“", '"').replace("”", '"').replace("’", "'")
        .replace("–", "-").replace("—", "-"))
    t = re.sub(r"[ \t]+", " ", t)
    t = re.sub(r"\s*\n\s*", "\n", t)
    t = t.strip(" \n")
    return {"text": t}

def load_data():
    print("Carga datasets")
    print("\tTiny Shakespeare")
    dataset = load_dataset("text", data_files={"raw": DATASET_PATH})

    # Dividir en train (90%) y test (10%)
    train_test = dataset["raw"].train_test_split(test_size=0.1, seed=42)

    # Dividir train en train (90%) y validation (10%)
    train_valid = train_test["train"].train_test_split(test_size=0.1, seed=42)

    # Reunir en un DatasetDict
    tinishakespeare = {
        "train": train_valid["train"],
        "validation": train_valid["test"],
        "test": train_test["test"]
    }

    print("\tWikitext-2")
    wikitext2 = load_dataset("Salesforce/wikitext", "wikitext-2-raw-v1")

    return tinishakespeare,wikitext2


def pre_clean_dataset(dataset):
    cleaned = DatasetDict({
        'train': dataset['train'].map(light_clean_fn, num_proc=4),
        'validation': dataset['validation'].map(light_clean_fn, num_proc=4),
        'test': dataset['test'].map(light_clean_fn, num_proc=4),
    })

    with train_path.open("w", encoding="utf-8") as f:
        for line in cleaned["train"]["text"]:
            if line.strip():
                f.write(line.strip() + "\n")

    with valid_path.open("w", encoding="utf-8") as f:
        for line in cleaned["validation"]["text"]:
            if line.strip():
                f.write(line.strip() + "\n")

    with test_path.open("w", encoding="utf-8") as f:
        for line in cleaned["test"]["text"]:
            if line.strip():
                f.write(line.strip() + "\n")

def read_datasets(only_test = False):

    if only_test:
        with test_path.open("r", encoding="utf-8") as f:
            test_text = f.read()
        return None,None, test_text

    with train_path.open("r", encoding="utf-8") as f:
        train_text = f.read()

    with valid_path.open("r", encoding="utf-8") as f:
        valid_text = f.read()

    with test_path.open("r", encoding="utf-8") as f:
        test_text = f.read()

    return train_text,valid_text,test_text


def tokenizador(dataset):
    if not os.path.exists(TOK_MODEL_PATH):
        print("Entrenamiento modelo BPE")
        # Aprende el vocabulario y como dividir en subpalabras
        spm.SentencePieceTrainer.Train(
            input=train_path, # Corpus de train, se pueden pasar varios
            model_prefix="./resources/models/bpe_model_shakespeare",        # genera prefijo modelos generados:  bpe_model_shakespeare.model y bpe_model_shakespeare.vocab
            vocab_size=10000,                 # 8k–32k para corpora pequeños, 32 k para wikitext2
            model_type="bpe",
            character_coverage=1.0,           # inglés
            byte_fallback=True,               # evita UNK en chars raros
            normalization_rule_name="nfkc",  # Normalización previa
            remove_extra_whitespaces=True, # colapsa espacios extra
            num_threads=os.cpu_count(),
            pad_id=0, unk_id=1, bos_id=2, eos_id=3
        )
    sp = spm.SentencePieceProcessor(model_file=TOK_MODEL_PATH)
    print("Tokenización del corpus de train, test y validation")
    tok_ids = sp.encode(dataset, out_type=int)
    print("Número total de tokens en el corpus:", len(tok_ids))
    return tok_ids, sp.vocab_size()


In [7]:

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads: int, d_model: int):

        super().__init__()
        assert d_model % num_heads == 0, "d_model debe ser divisible por num_heads"

        self.num_heads = num_heads
        self.d_model = d_model
        self.d_k = d_model // num_heads  # dimensión por cabeza

        # Proyecciones lineales para Q, K, V
        self.Wq = nn.Linear(d_model, d_model, bias=False)
        self.Wk = nn.Linear(d_model, d_model, bias=False)
        self.Wv = nn.Linear(d_model, d_model, bias=False)

        # Proyección final después de concatenar todas las cabezas
        self.Wo = nn.Linear(d_model, d_model, bias=False)

    def forward(self, x): # Que pasa con los datos al llamar al módulo
        """
        x: (B, T, d_model)
        Devuelve: (B, T, d_model)
        """
        B, T, _ = x.size()

        # 1. Proyecciones lineales
        Q = self.Wq(x)  # (B, T, d_model)
        K = self.Wk(x)  # (B, T, d_model)
        V = self.Wv(x)  # (B, T, d_model)

        # 2. Reorganizar en múltiples cabezas
        # (B, T, d_model) -> (B, num_heads, T, d_k)
        Q = Q.view(B, T, self.num_heads, self.d_k).transpose(1, 2)
        K = K.view(B, T, self.num_heads, self.d_k).transpose(1, 2)
        V = V.view(B, T, self.num_heads, self.d_k).transpose(1, 2)

        # 3. Calcular scores de atención
        # (B, num_heads, T, d_k) x (B, num_heads, d_k, T) -> (B, num_heads, T, T)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)

        # 4. Máscara causal (triangular superior a -inf)
        mask = torch.triu(torch.ones(T, T, device=x.device), diagonal=1).bool()
        scores = scores.masked_fill(mask, float('-inf'))

        # 5. Normalizar con softmax
        #with torch.amp.autocast(device_type="cuda",enabled=False):
        scores = scores.float()
        attn = torch.softmax(scores, dim=-1)  # (B, num_heads, T, T)

        # 6. Aplicar atención a V
        out = torch.matmul(attn, V)  # (B, num_heads, T, d_k)

        # 7. Recombinar heads
        out = out.transpose(1, 2).contiguous().view(B, T, self.d_model)  # (B, T, d_model)

        # 8. Proyección final
        out = self.Wo(out)  # (B, T, d_model)
        return out



# Add & norm


class NormLayer(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(normalized_shape))
        self.beta = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps

    def forward(self, X):
        # X: (batch, seq_len, hidden_dim)
        #with torch.amp.autocast(device_type="cuda",enabled=False): # Desactivo mixed precision manualmente pues es recomendable en la LayerNorm y como la he implementado yo, quiza no lo detecta automaticamente (solo reconoce capas nativas de pytorch) [35]
        X = X.float()
        mean = X.mean(dim=-1, keepdim=True)       # media por posición
        var = X.var(dim=-1, keepdim=True, unbiased=False)  # varianza
        X_hat = (X - mean) / torch.sqrt(var + self.eps)    # normaliza
        return self.gamma * X_hat + self.beta


class AddNorm(nn.Module):
    def __init__(self, norm_shape, dropout):
        super().__init__()
        self.dropout = nn.Dropout(dropout) # Para regularizar
        self.ln = NormLayer(norm_shape) # nn.LayerNorm(norm_shape)

    def forward(self, X, Y): # Y es la salida de la subcapa previa y X la entrada a la subcapa
        return self.ln(self.dropout(Y) + X) # Aplica add y luego layernorm

# FFNN

class FeedForward(nn.Module):
    def __init__(self,d_model, d_ff, dropout):
        super().__init__()
        # Se usan 3 capas densas, con dropout y activación GELU
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_ff)
        self.linear3 = nn.Linear(d_ff, d_model) # Ver si estas capas expanden y contraen
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
       x = self.linear1(x)
       x = self.gelu(x)
       x = self.dropout(x)
       x = self.linear2(x)
       x = self.gelu(x)
       x = self.dropout(x)
       x = self.linear3(x)
       return x

# Bloque 1 transformer decoder-only

class TransformerDecoderOnlyBlock(nn.Module):
    def __init__(self, num_heads, d_model, d_ff, dropout):
        super().__init__()
        self.mha = MultiHeadAttention(num_heads, d_model)
        self.addnorm1 = AddNorm(d_model, dropout)
        self.ffn = FeedForward(d_model, d_ff, dropout)
        self.addnorm2 = AddNorm(d_model, dropout)
        self.apply(self._init_weights) # Recorre las capas aplicando la función

    def _init_weights(self, m): # Inicialización de pesos: Xavier para MHA y FFNN y normal para embeddings
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.zeros_(m.bias) # Para evitar desplazamiento inicial arbitrario
        elif isinstance(m, nn.Embedding):
            nn.init.normal_(m.weight, mean=0.0, std=0.02)

    def forward(self,x):
        attention = self.mha(x)
        x = self.addnorm1(x, attention)
        ffn_out = self.ffn(x)
        x = self.addnorm2(x, ffn_out)
        return x


In [8]:

class miniGPT2(nn.Module):
    def __init__(self, vocab_size, d_model=512, num_heads=8, d_ff=2048, num_layers=6, context_len=256, dropout=0.1):
        super().__init__()
        self.tok_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = nn.Embedding(context_len, d_model)

        self.blocks = nn.ModuleList([
            TransformerDecoderOnlyBlock(num_heads, d_model, d_ff, dropout)
            for _ in range(num_layers)
        ])

        self.norm = NormLayer(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False)

    def forward(self, idx):
        B, T = idx.shape

        pos = torch.arange(T, device=idx.device).unsqueeze(0).expand(B, T)

        x = self.tok_emb(idx) + self.pos_emb(pos)
        for block in self.blocks:
            x = block(x)
        x = self.norm(x)
        logits = self.lm_head(x)
        return logits

In [10]:
os.makedirs("./resources/models", exist_ok=True)
os.makedirs("./resources/datasets", exist_ok=True)

print("Carpetas creadas correctamente.")


✅ Carpetas creadas correctamente.


In [11]:
!wget -q https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O ./resources/datasets/tinyshakespeare.txt
print("Dataset tinyshakespeare.txt descargado correctamente.")


Dataset tinyshakespeare.txt descargado correctamente.


In [13]:
class LMWindowDataset(torch.utils.data.Dataset):
    def __init__(self, tokens, context_len):
        self.toks = tokens
        self.ctx = context_len
    def __len__(self):
        return len(self.toks) - self.ctx # number of windows
    def __getitem__(self, i): # inputs:targets
        x = torch.tensor(self.toks[i:i+self.ctx], dtype=torch.long)
        y = torch.tensor(self.toks[i+1:i+self.ctx+1], dtype=torch.long)
        return x, y

In [15]:
print(os.getcwd())

/content


In [14]:
print("GPU available:", torch.cuda.is_available())

embedding_dim = 512              
context_len = 256
device = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_PATH = "./resources/models/gpt_model.pth"
BEST_MODEL_PATH = "./resources/models/best_gpt_model.pth"
########################
# Lectura de datasets
########################

tinishakespeare,wikitext2 = load_data()

#########################
# Limpieza y tokenización
##########################
pre_clean_dataset(tinishakespeare)

train_text,valid_text,test_text = read_datasets()

print(f"Longitud corpus train: {len(train_text)} caracteres")
print(f"Longitud corpus valid: {len(valid_text)} caracteres")
print(f"Longitud corpus test: {len(test_text)} caracteres")

train_ids,vocab_size = tokenizador(train_text)
val_ids,_ = tokenizador(valid_text)

#########################
# Preparación de batches <input, target>
##########################

# Se formatiza como <input, target>
 
print("Preparando DataLoader")
dataset = LMWindowDataset(train_ids, context_len=256)  # o 256
val_dataset = LMWindowDataset(val_ids, context_len=256)  # o 256

loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True,num_workers=os.cpu_count(),pin_memory=torch.cuda.is_available(),prefetch_factor=2)

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False,num_workers=os.cpu_count(),pin_memory=torch.cuda.is_available(),prefetch_factor=2)


########################
# Carga el modelo
########################

print("Se invoca el modelo")
model = miniGPT2(
    vocab_size=vocab_size,
    d_model=512,
    num_heads=8,
    d_ff=1024,
    num_layers=6,
    context_len=256,
    dropout=0.1
).to(device)

print("TRAIN")
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

num_epochs = 20
best_val_loss = float("inf")
best_loss = float("inf")
best_ppl = float("inf")

loss_fn = nn.CrossEntropyLoss()

#optimizer_sgd = torch.optim.SGD(model.parameters(), lr=1e-3)
#optimizer_rmsprop = torch.optim.RMSprop(model.parameters(),lr=1e-3, weight_decay=1e-2,momentum=0.9)

optimizer = torch.optim.AdamW(model.parameters(), lr=1.5e-4, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs*len(loader))



patience_limit = 10 # Para early stopping
patience_counter = 0
best_val_loss = float("inf")

train_losses, val_losses = [], []
train_ppls, val_ppls = [], []

#scaler = torch.amp.GradScaler("cuda") 

for epoch in range(num_epochs):
    '''
    Solo para validar en uno o pocos batches
    with profiler.profile(
        activities=[profiler.ProfilerActivity.CPU, profiler.ProfilerActivity.CUDA],
        profile_memory=True,
        record_shapes=True
    ) as prof:
    '''
    model.train()
    total_loss = 0.0
    torch.cuda.empty_cache()            
    print(f"Iteracciones: {len(loader)}")
    for num_batch, (batch_x, batch_y) in enumerate(loader, start=1):
        batch_x, batch_y = batch_x.to(device), batch_y.to(device) # todevice mueve a GPU si está disponible, es necesario pues le modelo está en GPU también.
        optimizer.zero_grad(set_to_none=True) # Limpia los gradientes previos. Pues en pytorch se acumulan por defecto y al llamar a backward se suman a los previos, es decir, se estarían combinando gradientes de varios batches. Lo pone a none para ahorrar memoria con set_to_none.

        #with torch.amp.autocast("cuda"): # Decide de forma segura que algunas operaciones se hagan en FP16 para ahorrar memoria y acelerar GPU
        logits = model(batch_x) # Salida del modelo [batch_size, seq_len, vocab_size]. Cada posición de la secuencia tiene una distribución sobre el vocabulario.
            # Devuelve, para cada token de entrada, un vector de tamaño vocab_size con valores reales. En PyTorch, nn.CrossEntropyLoss()  ya incluye internamente el softmax
        loss = loss_fn(
            logits.view(-1, logits.size(-1)),
            batch_y.view(-1) # Batch_y tiene: [batch_size, seq_len], esto lo aplana en [batch_size * seq_len] para cross entropy
        )
        loss.backward()
        #scaler.scale(loss).backward() # Escala y calcula el gradiente de los pesos del modelo (en que dirección cambio el peso para reducir la pérdida). Scale multiplica la pérdida por un factor grande antes de hacer el backward. Los pesos aún no cambian. 
        
        # Gradient Clipping para estabilidad [35]: limita el tamaño máximo que puede tener el conjunto de gradientes antes de actualizar los pesos.El clipping no cambia la dirección del gradiente (sigue apuntando hacia la misma mejora), solo reduce su magnitud para que no provoque saltos gigantes en los pesos.
        #scaler.unscale_(optimizer) # En AMP, los gradientes están temporalmente amplificados (por GradScaler).Se desescalan para no recortar valores falsos.
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        #scaler.step(optimizer) # usa los gradientes calculados para ajustar los pesos. Sin GradScaler sería: optimizer.step(). El scaler primero desescala los gradientes si aún no se ha hecho (los divide por el mismo factor que usó en scale) y luego llama a optimizer.step().
        #scaler.update() # Se encarga de ajustar el factor de escalado de la pérdida para la próxima iteración.

        total_loss += loss.item()

        if num_batch % 100 == 0:
            current_lr = scheduler.get_last_lr()[0]
            print(f"  [Batch {num_batch}] Loss: {loss.item():.4f} | LR: {current_lr:.6e}")

    avg_loss = total_loss / len(loader)
    train_ppl = math.exp(avg_loss) # Perplejidad: Si baja es que comprende mejor los datos. Es el exponente de la entropía
    # Validación para implementar early stopping y guardar mejor modelo

    val_loss, val_ppl = evaluate_ppl(model, val_loader, loss_fn, device)
    print(f"[Epoch {epoch+1}] Train Loss: {avg_loss:.4f} | Train PPL: {train_ppl:.2f} | Val PPL: {val_ppl:.2f}")
    train_losses.append(avg_loss)
    val_losses.append(val_loss)
    train_ppls.append(train_ppl)
    val_ppls.append(val_ppl)


    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_loss = avg_loss
        best_ppl = train_ppl
        patience_counter = 0
        torch.save(model.state_dict(), BEST_MODEL_PATH)

    else:
        patience_counter += 1
        if patience_counter >= patience_limit:
            print(f"Early stopping en epoch: {epoch+1}")
            break


print("Entrenamiento finalizado.")
torch.save(model, MODEL_PATH)

print(f"Best Train Loss: {best_loss:.4f} | Best Train PPL: {best_ppl:.2f}")

plt.figure(figsize=(10,5))
plt.plot(train_ppls, label="Train PPL")
plt.plot(val_ppls, label="Validation PPL")
plt.xlabel("Época")
plt.ylabel("Perplexity")
plt.title("Evolución de Perplexity durante entrenamiento")
plt.legend()
plt.grid(True)
plt.savefig('./resources/imagenes/resultado_entrenamiento_v1.pdf')

GPU available: True
Carga datasets
	Tiny Shakespeare
	Wikitext-2
Longitud corpus train: 896135 caracteres
Longitud corpus valid: 100448 caracteres
Longitud corpus test: 111570 caracteres
Tokenización del corpus de train, test y validation
Número total de tokens en el corpus: 273905
Tokenización del corpus de train, test y validation
Número total de tokens en el corpus: 31059
Preparando DataLoader
Se invoca el modelo
TRAIN
Iteracciones: 4276
  [Batch 100] Loss: 6.2542
  [Batch 200] Loss: 6.2575
  [Batch 300] Loss: 6.2547


KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/transformer_colab/results
!cp -r /content/resources /content/drive/MyDrive/transformer_colab/

